In [1]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import  plotly as py
import pandas as pd

# Total population, N.
N = 1000000
# Initial number of infected and recovered individuals, I0 and R0.
IA1_0, IP1_0, IM1_0, IH1_0, IC1_0, Ra1_0, Rb1_0, D1_0 = 1, 1, 1, 1, 1, 0, 0, 0
IA2_0, IP2_0, IM2_0, IH2_0, IC2_0, Ra2_0, Rb2_0, D2_0 = 1, 1, 1, 1, 1, 0, 0, 0
E1_0 = np.arange(2, 500, 1)[0]
E2_0 = np.arange(2, 500, 1)[0]
# Everyone else, S0, is susceptible to infection initially.
S0 = N - (E1_0 + E2_0)

# Parameters
β1 =  np.arange(0.1428, 1.5, 0.1)[0]
βx = 1.5 #3, 6, 9
β2 =  np.arange(0.1428, 1.5, 0.1)[0] * (1 + βx)
print(β1, β2)
α = 2.0    #-set to quickly move S --> R1 at lockdown start
σ = np.arange(0.16, 0.5, 0.1)[0] # inverse of 2-6 days
ρ = np.arange(0.25, 0.5, 0.1)[0] # 25-50% of cases are asymptomatic
γA = np.arange(0.125, 0.33, 0.1)[0] # gammaA inverse of 4-14 days recovery
γM = np.arange(0.125, 0.33, 0.1)[0]
γH = np.arange(0.125, 0.33, 0.1)[0]
γC = np.arange(0.125, 0.33, 0.1)[0]
δ1 = np.arange(0.05, 1, 0.05)[0] #inverse of 1-1_0 days - modified to 1-20 days
δ2 = np.arange(0.06, 0.25, 0.05)[0] #inverse of 4-15 days
δ3 = np.arange(0.09, 1, 0.05)[0] #inverse of 1-11 days
m = np.arange(0.08, 0.25, 0.1)[0]
ξ = np.arange(0.1, 0.3, 0.1)[0] #proportion of symptomatic cases undetected
x1 = np.arange(0.05, 0.3, 0.1)[0]
x2 = np.arange(0.2, 0.3, 0.1)[0]
x3 = np.arange(0.2, 0.8, 0.1)[0]
d = 0.9 #np.arange(0.1, 0.4, 0.1)[0] #1-[0.58 0.85]
q = 0
λ = 4.6849



       # A grid of time points (in days)
t = np.linspace(0, 365, 365)

# The SIR model differential equations.
def deriv(y, t, N, β1, β2, α, σ, ρ, γA, γM, γH, γC, δ1, δ2, δ3, m, ξ, x1, x2, x3, d, q, λ):
    S, E1, IA1, IP1, IM1, IH1, IC1, Ra1, Rb1, D1, E2, IA2, IP2, IM2, IH2, IC2, Ra2, Rb2, D2 = y
    dSdt = -d* β1 *S*(1-q)*(IA1+IP1+IM1+IH1+IC1)/N
    dE1dt = d*β1*S*(1-q)*(IA1+IP1+IM1+IH1+IC1)/N - σ*ρ*E1 - σ*(1-ρ)*E1
    dIA1dt = σ*ρ*E1 - γA*IA1
    dIP1dt = σ*(1-ρ)*E1 - (δ1*IP1)
    dIM1dt = δ1*IP1 - x1*δ2*IM1 - (1-x1)*γM*IM1
    dIH1dt = x1*δ2*IM1 - x2*δ3*IH1 - (1-x2)*γH*IH1
    dIC1dt = x2*δ3*IH1 - (1-x3)*γC*IC1 - x3*m*IC1
    dRb1dt = 0
    dRa1dt = γA*IA1 + (1-x1)*γM*IM1 + (1-x2)*γH*IH1 + (1-x3)*γC*IC1
    dD1dt  = x3*m*IC1

    dE2dt = d*β2*S*(1-q)*(IA2+IP2+IM2+IH2+IC2)/(N) - σ*ρ*E2 - σ*(1-ρ)*E2
    dIA2dt = σ*ρ*E2 - γA*IA2
    dIP2dt = σ*(1-ρ)*E2 - (δ1*IP2)
    dIM2dt = δ1*IP2 - x1*δ2*IM2 - (1-x1)*γM*IM2
    dIH2dt = x1*δ2*IM2 - x2*δ3*IH2 - (1-x2)*γH*IH2
    dIC2dt = x2*δ3*IH2 - (1-x3)*γC*IC2 - x3*m*IC2
    dRb2dt = 0
    dRa2dt = γA*IA2 + (1-x1)*γM*IM2 + (1-x2)*γH*IH2 + (1-x3)*γC*IC2
    dD2dt  = x3*m*2*IC2
    # dE2dt = 0
    # dIA2dt = 0
    # dIP2dt = 0
    # dIM2dt = 0
    # dIH2dt = 0
    # dIC2dt = 0
    # dRb2dt = 0
    # dRa2dt = 0
    # dD2dt  = 0
    return dSdt, dE1dt, dIA1dt, dIP1dt, dIM1dt, dIH1dt, dIC1dt, dRa1dt, dRb1dt, dD1dt, dE2dt*0.05, dIA2dt, dIP2dt, dIM2dt, dIH2dt, dIC2dt, dRa2dt, dRb2dt, dD2dt

# Initial conditions vector
y0 = S0, E1_0, IA1_0, IP1_0, IM1_0, IH1_0, IC1_0, Ra1_0, Rb1_0, D1_0, E2_0, IA2_0, IP2_0, IM2_0, IH2_0, IC2_0, Ra2_0, Rb2_0, D2_0
# Integrate the SIR equations over the time grid, t.
ret = odeint(deriv, y0, t, args=(N, β1, β2, α, σ, ρ, γA, γM, γH, γC, δ1, δ2, δ3, m, ξ, x1, x2, x3, d, q, λ))
S, E1, IA1, IP1, IM1, IH1, IC1, Ra1, Rb1, D1, E2, IA2, IP2, IM2, IH2, IC2, Ra2, Rb2, D2 = ret.T

# df = pd.DataFrame()
# # df['t'] = t.astype(np.int64)
# df['I_1.5x'] = IA1+IP1+IM1+IH1+IC1 + IA2+IP2+IM2+IH2+IC2
# df['H_1.5x'] = IH1 + IC1 + IH2 + IC2


# Create traces
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.01, horizontal_spacing=0.05)
fig.add_trace(go.Scatter(x=t, y=S, mode='lines', name='S', line = dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=E1, mode='lines', name='E', line = dict(color='orange')), row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=IA1, mode='lines', name='IA1', line = dict(color='crimson')), row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=IP1, mode='lines', name='IP1', line = dict(color='red')),  row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=IM1, mode='lines', name='IM1', line = dict(color='red', dash='dash')),  row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=IH1, mode='lines', name='IH1', line = dict(color='red', dash='dot')),  row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=IC1, mode='lines', name='IC1', line = dict(color='red', dash='dashdot')),  row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=Ra1, mode='lines', name='Ra1', line = dict(color='green')), row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=D1, mode='lines', name='D1', line = dict(color='black')), row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=IA2, mode='lines', name='IA2', line = dict(color='crimson')),  row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=IP2, mode='lines', name='IP2', line = dict(color='red')), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=IM2, mode='lines', name='IM2', line = dict(color='red', dash='dash')), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=IH2, mode='lines', name='IH2', line = dict(color='red', dash='dot')), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=IC2, mode='lines', name='IC2', line = dict(color='red', dash='dashdot')),  row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=Ra2, mode='lines', name='Ra2', line = dict(color='green', dash='dash')), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=D2, mode='lines', name='D2', line = dict(color='black')), row=2, col=1)
# xaxis = dict(dtick=1_0), yaxis = dict(dtick=N*0.1),
fig.update_layout(showlegend=True,  autosize=True, width=1_024*1.5, height=800)
py.offline.plot(fig, filename='plot' + str(βx) + '.html')

0.1428 0.35700000000000004


'plot1.5.html'